### Comparison of results from validate_drp and SPORK on the same dataset.

In particular, this comparison is run on HSC RC2.

In [ ]:
import random
import numpy as np
from astropy.table import Table
from astropy import units as u
from astropy.table import hstack, Column
import matplotlib.pyplot as plt
import json
import os

import lsst.verify
import lsst.geom as geom
import lsst.daf.persistence as dafPersist
from lsst.verify.gen2tasks import register, MetricsControllerTask
from lsst.verify.tasks import MetricTask
import lsst.daf.butler as dafButler

In [ ]:
# Output from a run of validate_drp:
validate_job_g9813 = '/datasets/hsc/repo/rerun/RC/w_2020_34/DM-26441/validateDrp/matchedVisitMetrics/9813/HSC-G/matchedVisit_HSC-G.json'
validate_job_r9813 = '/datasets/hsc/repo/rerun/RC/w_2020_34/DM-26441/validateDrp/matchedVisitMetrics/9813/HSC-R/matchedVisit_HSC-R.json'
validate_job_i9813 = '/datasets/hsc/repo/rerun/RC/w_2020_34/DM-26441/validateDrp/matchedVisitMetrics/9813/HSC-I/matchedVisit_HSC-I.json'

The validate_drp results come from executing something similar to the following commands (shown here running on the v20_0_0_rc1 processing of the RC2 dataset):

matchedVisitMetrics.py /datasets/hsc/repo/rerun/RC/v20_0_0_rc1/DM-25349 --output /project/jcarlin/mvm_w_25 --config instrumentName='HSC' datasetName='HSC-RC2' --id tract=9615 filter='HSC-G^HSC-R^HSC-I'

matchedVisitMetrics.py /datasets/hsc/repo/rerun/RC/v20_0_0_rc1/DM-25349 --output /project/jcarlin/mvm_w_25/9813 --config instrumentName='HSC' datasetName='HSC-RC2' --id tract=9813 filter='HSC-G^HSC-R^HSC-I'

matchedVisitMetrics.py /datasets/hsc/repo/rerun/RC/v20_0_0_rc1/DM-25349 --output /project/jcarlin/mvm_w_25/9697 --config instrumentName='HSC' datasetName='HSC-RC2' --id tract=9697 filter='HSC-G^HSC-R^HSC-I'

In [ ]:
# Read in the .json saved by each validate_drp run:
with open(validate_job_g9813) as f:
    job_g9813 = lsst.verify.Job.deserialize(**json.load(f))
with open(validate_job_r9813) as f:
    job_r9813 = lsst.verify.Job.deserialize(**json.load(f))
with open(validate_job_i9813) as f:
    job_i9813 = lsst.verify.Job.deserialize(**json.load(f))

In [ ]:
# Show a metric report in the notebook (use "spec_tags" to specify design, stretch, or minimum req level):
job_g9813.report(spec_tags=['design']).show()

In [ ]:
# Extract the measurements from the JSON:
meas_g9813 = job_g9813.measurements.json
meas_r9813 = job_r9813.measurements.json
meas_i9813 = job_i9813.measurements.json

In [ ]:
# Compile the measurements into arrays:

gen2_names_g9813 = []
gen2_vals_g9813 = []
gen2_units_g9813 = []

for met in meas_g9813:
    # print(met['metric'], met['value'], met['unit'])
    gen2_names_g9813.append(met['metric'])
    gen2_vals_g9813.append(met['value'])
    gen2_units_g9813.append(met['unit'])
    
gen2_names_r9813 = []
gen2_vals_r9813 = []
gen2_units_r9813 = []

for met in meas_r9813:
    # print(met['metric'], met['value'], met['unit'])
    gen2_names_r9813.append(met['metric'])
    gen2_vals_r9813.append(met['value'])
    gen2_units_r9813.append(met['unit'])
    
gen2_names_i9813 = []
gen2_vals_i9813 = []
gen2_units_i9813 = []

for met in meas_i9813:
    # print(met['metric'], met['value'], met['unit'])
    gen2_names_i9813.append(met['metric'])
    gen2_vals_i9813.append(met['value'])
    gen2_units_i9813.append(met['unit'])

In [ ]:
# Make the arrays into astropy tables:
tab_gen2_g9813 = Table([gen2_names_g9813, gen2_vals_g9813, gen2_units_g9813], names=['metric', 'value', 'units'], dtype=(str, 'f2', str))
tab_gen2_r9813 = Table([gen2_names_r9813, gen2_vals_r9813, gen2_units_r9813], names=['metric', 'value', 'units'], dtype=(str, 'f2', str))
tab_gen2_i9813 = Table([gen2_names_i9813, gen2_vals_i9813, gen2_units_i9813], names=['metric', 'value', 'units'], dtype=(str, 'f2', str))

### Now get the Gen 3 butler results from running SPORK on RC2

In [ ]:
repo = '/project/hsc/gen3repo/rc2w34_ssw36/'
config = os.path.join(repo,'butler.yaml')
try: butler_gen3 = dafButler.Butler(config=config)
except ValueError as e: print(e)

In [ ]:
registry = butler_gen3.registry

In [ ]:
# To see what collections are in the repo:
for c in registry.queryCollections():
    print(c)

In [ ]:
# To see what dataset types are (potentially) available:
dstypes = []
for x in registry.queryDatasetTypes():
    print(x)
    dstypes.append(x)

In [ ]:
for d in dstypes:
    if 'match' in d.name:
        print(d)

In [ ]:
# These are the names of the metrics we plan to extract from the repo:
metricnames = ['PA1', 'PA2_design_gri', 'PF1_design_gri', 'AM1', 'AM2', 'AM3',\
               'AD1_design', 'AD2_design', 'AD3_design', 'AF1_design', 'AF2_design',\
               'AF3_design', 'TE1', 'TE2', 'AB1']
#metricnames = ['PA1', 'PA2_design_gri', 'PF1_design_gri', 'AM1', 'AM2', 'AM3', 'AD1_design', 'AF1_design',
#               'AD2_design', 'AF2_design', 'AD3_design', 'AF3_design', 'TE1', 'TE2', 'AB1_design']

In [ ]:
collectionName = 'kbechtol/svv_9813_gri_matched'
#collectionName = 'jcarlin/t9813_HSC-R_only'

In [ ]:
def get_metric_meas(collectionName, filt, tract, metrics):
    
    metric_agg_vals = []
    metric_unit = []
    metric_filter = []
    metric_name = []
    metric_ids = []
    metric_meas = []

    did={'tract':str(tract), 'abstract_filter':filt, 'instrument':'HSC', 'skymap':'hsc_rings_v1'}

    for metric in metrics:
        dataset = f"metricvalue_summary_validate_drp_{metric}"
#        dataset = f"metricvalue_summary_validate_drp_{metric}"
        query = registry.queryDatasets(datasetType=dataset, collections=[collectionName], dataId=did)
        for meas in query:            
            metric_tmp = butler_gen3.get(meas, collections=[collectionName])
            metric_agg_vals.append(metric_tmp.quantity.value)
            metric_unit.append(metric_tmp.quantity.unit)
            metric_name.append(metric)
            metric_ids.append(meas)
            metric_meas.append(metric_tmp)
            metric_filter.append(meas.dataId['abstract_filter'])
    
    tab_metrics = Table([metric_name, metric_unit, metric_filter, metric_agg_vals], 
                        names=('metric', 'unit', 'filter', str(tract)+'_gen3'), dtype=(str, str, str, 'f2'))
        
    return(tab_metrics)

In [ ]:
#dataset1 = "metricvalue_summary_validate_drp_PA1"
#dataset2 = "metricvalue_Mean_validate_drp_PA1"
#dataset3 = "metricvalue_validate_drp_PA1"
dataset3 = "matchedCatalog"
did={'tract':9813, 'abstract_filter':'r', 'instrument':'HSC', 'skymap':'hsc_rings_v1', 'patch':59}
#metric_tmp1 = butler_gen3.get(dataset1, dataId=did, collections=[collectionName])
#metric_tmp2 = butler_gen3.get(dataset2, dataId=did, collections=[collectionName])
metric_tmp3 = butler_gen3.get(dataset3, dataId=did, collections=[collectionName])

In [ ]:
#print(metric_tmp1, metric_tmp2, metric_tmp3)
#print(did)
len(set(metric_tmp3['object']))

In [ ]:
#for s in metric_tmp3.schema:
#    if 'flag' in s.field.getName():
#        print(s.field.getName())

In [ ]:
# Extract astropy tables of the metric values by calling the function defined above:
tab_gen3_g9813 = get_metric_meas(collectionName, 'g', 9813, metricnames)
tab_gen3_r9813 = get_metric_meas(collectionName, 'r', 9813, metricnames)
tab_gen3_i9813 = get_metric_meas(collectionName, 'i', 9813, metricnames)

In [ ]:
tab_gen3_r9813

Loop over the names of the metrics, picking out the corresponding measurements from Gen2 and Gen3 (and possibly from tracts/filters separately), compiling them into a table together. (Later, we may want to add in the specs?)

In [ ]:
tab_metrics_g9813 = tab_gen3_g9813.copy()
matchcol=np.zeros(len(tab_metrics_g9813['9813_gen3']))

for i in range(len(tab_metrics_g9813['metric'])):
    name = tab_metrics_g9813['metric'][i]
    find_in_tab2 = np.where(np.char.find(tab_gen2_g9813['metric'], name) >= 0)

    if np.size(find_in_tab2) > 0:
        matchcol[i] = tab_gen2_g9813['value'][find_in_tab2[0]]
    else:
        matchcol[i] = np.nan
    
tab_metrics_g9813_all = hstack([tab_metrics_g9813, Column(matchcol, name='9813_gen2', dtype='f2')])

In [ ]:
tab_metrics_r9813 = tab_gen3_r9813.copy()
matchcol=np.zeros(len(tab_metrics_r9813['9813_gen3']))

for i in range(len(tab_metrics_r9813['metric'])):
    name = tab_metrics_r9813['metric'][i]
    find_in_tab2 = np.where(np.char.find(tab_gen2_r9813['metric'], name) >= 0)

    if np.size(find_in_tab2) > 0:
        matchcol[i] = tab_gen2_r9813['value'][find_in_tab2[0]]
    else:
        matchcol[i] = np.nan
    
tab_metrics_r9813_all = hstack([tab_metrics_r9813, Column(matchcol, name='9813_gen2', dtype='f2')])

In [ ]:
tab_metrics_i9813 = tab_gen3_i9813.copy()
matchcol=np.zeros(len(tab_metrics_i9813['9813_gen3']))

for i in range(len(tab_metrics_i9813['metric'])):
    name = tab_metrics_i9813['metric'][i]
    find_in_tab2 = np.where(np.char.find(tab_gen2_i9813['metric'], name) >= 0)

    if np.size(find_in_tab2) > 0:
        matchcol[i] = tab_gen2_i9813['value'][find_in_tab2[0]]
    else:
        matchcol[i] = np.nan
    
tab_metrics_i9813_all = hstack([tab_metrics_i9813, Column(matchcol, name='9813_gen2', dtype='f2')])

In [ ]:
# Combine them all into a single table:
tab_gr_tmp = hstack([tab_metrics_g9813_all, tab_metrics_r9813_all['filter','9813_gen3', '9813_gen2']])
tab_metrics_9813_all = hstack([tab_gr_tmp, tab_metrics_i9813_all['filter','9813_gen3', '9813_gen2']])

In [ ]:
tab_metrics_9813_all

In [ ]:
# Write out a Latex table
#tab_metrics_9813_all.write('compare_metrics_gen2and3_tract9813_RC2.tex', format='ascii.latex')

### Plot the metric measurements per patch

In [ ]:
dimensions = registry.queryDataIds('patch',
                                      dataId={'tract':9813, 'abstract_filter':'r', 'instrument':'HSC', 'skymap':'hsc_rings_v1'},
                                      collections=[collectionName],datasets='metricvalue_summary_validate_drp_PA1')

In [ ]:
patch_tmp = []
for d in dimensions:
    patch_tmp.append(d['patch'])

In [ ]:
patch = np.unique(patch_tmp)

In [ ]:
did={'tract':9813, 'abstract_filter':'r', 'instrument':'HSC', 'skymap':'hsc_rings_v1', 'patch':59}
sm = butler_gen3.get('deepCoadd_skyMap', collections=['RC2/w_2020_34/DM-26441/remainder'],dataId=did)
tract_info = sm.generateTract(9813)
wcs = tract_info.getWcs()

patchlist = []
for patch in tract_info:
    # print(patch)
    patchlist.append(patch)

In [ ]:
# patchlist

In [ ]:
### Warning - this takes a long time to run!

metric_agg_vals = np.zeros((len(patchlist),len(metricnames)))
metric_agg_vals[:] = np.nan
#metric_agg_vals = []
metric_patch = []
metric_ra = []
metric_dec = []

# For some reason patch 0, 8, 9, 63, 72 don't exist, so I had to skip them. Also note that butler_gen3.datasetExists fails on
#   these patches, so I can't even use that to check for existence. Instead use try/except:
for pch in patchlist:
    p = tract_info.getSequentialPatchIndex(pch)
    metric_patch.append(p)
#    if p not in [8, 9, 63, 72]:
    did={'tract':9813, 'abstract_filter':'r', 'instrument':'HSC', 'skymap':'hsc_rings_v1', 'patch':p}
    try:
        coadd = butler_gen3.get('deepCoadd_calexp', collections=['RC2/w_2020_34/DM-26441/remainder'],dataId=did)

        for i in range(len(metricnames)):
            metric = metricnames[i]
            dataset = f"metricvalue_validate_drp_{metric}"
    
            query = registry.queryDatasets(datasetType=dataset, collections=[collectionName], dataId=did)
            for meas in query:            
                metric_tmp = butler_gen3.get(meas, collections=[collectionName])
                metric_agg_vals[p,i]=metric_tmp.quantity.value

                if i == 0:
                #try:
                    #coadd = butler_gen3.get('deepCoadd_calexp', collections=['RC2/w_2020_34/DM-26441/remainder'],dataId=did)
                    wcs = coadd.getWcs()
                    xy0 = wcs.pixelToSky(geom.Point2D(coadd.getXY0()))
                    metric_ra.append(xy0.getRa().asDegrees())
                    metric_dec.append(xy0.getDec().asDegrees())
                
            #metric_agg_tmp = np.array(metric_agg_tmp)
            #metric_agg_vals.append(metric_agg_tmp)
    except:
        #metric_agg_tmp = np.zeros(len(metricnames))
        #metric_agg_tmp[:] = np.nan
        #metric_agg_vals[p,:] = metric_agg_tmp
        metric_ra.append(999.9999)
        metric_dec.append(999.9999)
        print('Patch ',p,' not found. Skipping.')

    
tab_metrics_perpatch_all = Table([metric_patch, metric_ra, metric_dec, metric_agg_vals],
                                 names=('patch', 'ra', 'dec', 'metrics'))

In [ ]:
metric_agg_vals[55]

In [ ]:
#did={'tract':9813, 'abstract_filter':'r', 'instrument':'HSC', 'skymap':'hsc_rings_v1', 'patch':8}
#print(did)
#butler_gen3.datasetExists('deepCoadd_calexp', collections=['RC2/w_2020_34/DM-26441/remainder'], dataId=did)

In [ ]:
# Get rid of the ones with no data:

tab_metrics_perpatch = tab_metrics_perpatch_all[tab_metrics_perpatch_all['ra'] < 400]

In [ ]:
tab_metrics_perpatch

In [ ]:
params = {
   'axes.labelsize': 28,
   'font.size': 24,
   'legend.fontsize': 14,
   'xtick.major.width': 3,
   'xtick.minor.width': 2,
   'xtick.major.size': 12,
   'xtick.minor.size': 6,
   'xtick.direction': 'in',
   'xtick.top': True,
   'lines.linewidth':3,
   'axes.linewidth':3,
   'axes.labelweight':3,
   'axes.titleweight':3,
   'ytick.major.width':3,
   'ytick.minor.width':2,
   'ytick.major.size': 12,
   'ytick.minor.size': 6,
   'ytick.direction': 'in',
   'ytick.right': True,
   'figure.figsize': [10, 8]
   }

plt.rcParams.update(params)

metric_to_plot = metricnames[0] #'PA1' # set this to any of the metrics in "metricnames"
find_metric_name = np.where(np.char.find(metricnames, metric_to_plot) >= 0)

plt.scatter(tab_metrics_perpatch['ra'], tab_metrics_perpatch['dec'], c=tab_metrics_perpatch['metrics'][:,find_metric_name], 
            cmap='rainbow', s=60)
#plt.xlim(216.4,216.1)
#plt.ylim(0.742,0.746)
plt.gca().invert_xaxis()
plt.xlabel(r'RA (deg)')
plt.ylabel(r'Dec (deg)')
plt.minorticks_on()
plt.colorbar(label=metricnames[find_metric_name[0][0]])
plt.show()

Check the actual matched catalog:

In [ ]:
dataset = "matchedCatalog"
patch = 3
did={'tract':9813, 'abstract_filter':'r', 'instrument':'HSC', 'skymap':'hsc_rings_v1', 'patch':patch}
mch_cat = butler_gen3.get(dataset, dataId=did, collections=[collectionName])
metric_tmp3 = butler_gen3.get('metricvalue_validate_drp_PA1', dataId=did, collections=[collectionName])

In [ ]:
objs = np.unique(mch_cat['object'])

In [ ]:
rms_arr = []

for obj in objs:
    pickobj = (mch_cat['object'] == obj) & (mch_cat['base_ClassificationExtendedness_value'] < 0.5) &\
              (mch_cat['base_PsfFlux_snr'] > 50.0) & (mch_cat['base_PixelFlags_flag_saturated'] == False) &\
              (mch_cat['base_PixelFlags_flag_cr'] == False) & (mch_cat['base_PixelFlags_flag_bad'] == False) &\
              (mch_cat['base_PixelFlags_flag_edge'] == False)
    if len(mch_cat[pickobj]) > 1:
        mags_tmp = mch_cat[pickobj]['base_PsfFlux_mag']
        # mch_cat[pickobj]['base_ClassificationExtendedness_value']
        diffs = mags_tmp - np.mean(mags_tmp)
        rms = np.sqrt(np.mean((diffs)**2))*1000.0
        rms_arr.append(rms)
#        diffs = mch_cat[pickobj[0]]['base_PsfFlux_mag'] - mch_cat[pickobj]['base_PsfFlux_mag']
        print(rms, len(diffs), mch_cat[pickobj]['base_PsfFlux_mag'])

print('median: ',np.median(rms_arr),' Meas: ',metric_tmp3)
#        print(np.sqrt(np.mean((diffs-np.mean(diffs))**2))*1000.0, len(diffs), np.mean(diffs), mch_cat[pickobj]['base_PsfFlux_mag'])

In [ ]:
plt.hist(rms_arr, bins=np.arange(0, 50, 1))
plt.show()

**Measure PA1 directly for all patches**

In [ ]:
# I set the RA, Dec for patches with no data to 999.999. Filter those out:
filt = np.where(tab_metrics_perpatch_all['ra'] < 400)
patches_tmp = np.array(metric_patch)
patches = patches_tmp[filt]

In [ ]:
def cal_RMS(x):
    return np.sqrt(np.mean((x-np.mean(x))**2))

In [ ]:
dataset = "matchedCatalog"

pa1_direct = []
pa1_mpt = []
nstars_patch = []
magdiffs_all = []

for patch in patches:
    print('patch: ',patch)
    did={'tract':9813, 'abstract_filter':'r', 'instrument':'HSC', 'skymap':'hsc_rings_v1', 'patch':patch}
    mch_cat_tmp = butler_gen3.get(dataset, dataId=did, collections=[collectionName])
    mch_cat1 = mch_cat_tmp.copy(deep=True)
    qual_cuts = (mch_cat1['base_ClassificationExtendedness_value'] < 0.5) &\
                (mch_cat1['base_PsfFlux_snr'] > 50.0) & (mch_cat1['base_PixelFlags_flag_saturated'] == False) &\
                (mch_cat1['base_PixelFlags_flag_cr'] == False) & (mch_cat1['base_PixelFlags_flag_bad'] == False) &\
                (mch_cat1['base_PixelFlags_flag_edge'] == False)
    mch2 = mch_cat1[qual_cuts].copy(deep=True).asAstropy()
    mch_cat = mch2.to_pandas()    
    metric_tmp3 = butler_gen3.get('metricvalue_validate_drp_PA1', dataId=did, collections=[collectionName])

    rms_arr = []
    # count = 0

    mch_cat_agg = mch_cat.groupby('object').agg({'base_PsfFlux_mag':['mean',cal_RMS,'count']})
    ok_agg = (mch_cat_agg.base_PsfFlux_mag['count'] > 1)
    rms_arr.append(1000.0*mch_cat_agg.base_PsfFlux_mag['cal_RMS'][ok_agg])
    nstars_patch.append(len(mch_cat_agg.base_PsfFlux_mag['cal_RMS'][ok_agg]))

    pa1_direct.append(np.median(rms_arr))
    pa1_mpt.append(metric_tmp3.quantity.value)

In [ ]:
tab_compare = Table([patches, pa1_direct, pa1_mpt, nstars_patch],\
                    names=('patch', 'PA1_direct', 'PA1_MPT', 'nstars_patch'), dtype=(int, 'f2', 'f2', int))

In [ ]:
tab_compare

In [ ]:
#tab_compare.write('RC2_tract9813_PA1_compare.fits')

In [ ]:
plt.scatter(tab_compare['nstars_patch'],\
            100.0*(tab_compare['PA1_direct']-tab_compare['PA1_MPT'])/tab_compare['PA1_direct'],\
            c=tab_compare['PA1_MPT'], s=50, cmap='gist_rainbow')
plt.xlabel(r'N$_{\rm stars}$')
plt.ylabel('Percent difference')
plt.hlines(0,0,300,linestyle='--')
plt.xlim(0,140)
plt.colorbar(label='PA1_MPT (mmag)')
plt.minorticks_on()
plt.savefig('PA1_comp_RC2.png')
plt.show()

In [ ]:
plt.hist(tab_compare['PA1_direct'], bins=np.arange(0,50,2), color='Red', label='direct measurement', histtype='step', linewidth=4)
plt.hist(tab_compare['PA1_MPT'], bins=np.arange(0,50,2), color='Black', linestyle='--', label='metrics-pipeline-task', histtype='step', linewidth=4)
plt.vlines(np.nanmedian(tab_compare['PA1_direct']),0,17,linestyle=':',color='Red',label='median: '+str(np.nanmedian(tab_compare['PA1_direct'])))
plt.vlines(np.nanmedian(tab_compare['PA1_MPT']),0,17,linestyle=':',color='Black',label='median: '+str(np.nanmedian(tab_compare['PA1_MPT'])))
plt.xlabel('PA1 (mmag)')
plt.ylabel('N')
plt.ylim(0,15)
plt.legend()
plt.minorticks_on()
plt.savefig('PA1_comp_RC2_tract9813_hist.png')
plt.show()

In [ ]:
from lsst.afw.table import GroupView

In [ ]:
dataset3 = "matchedCatalog"

#mch_cat_tmp = []

count=0
for p in patches[68:70]:

    did={'tract':9813, 'abstract_filter':'r', 'instrument':'HSC', 'skymap':'hsc_rings_v1', 'patch':p}
    mch_cat_tmp0 = butler_gen3.get(dataset3, dataId=did, collections=[collectionName])
    if count==0:
        mch_cat_tmp = mch_cat_tmp0.copy()
    else:
        mch_cat_tmp.append(mch_cat_tmp0)

gv = GroupView.build(mch_cat_tmp)

In [ ]:
qual_cuts = (mch_cat_tmp['base_ClassificationExtendedness_value'] < 0.5) &\
            (mch_cat_tmp['base_PsfFlux_snr'] > 50.0) & (mch_cat_tmp['base_PixelFlags_flag_saturated'] == False) &\
            (mch_cat_tmp['base_PixelFlags_flag_cr'] == False) & (mch_cat_tmp['base_PixelFlags_flag_bad'] == False) &\
            (mch_cat_tmp['base_PixelFlags_flag_edge'] == False)

mch_cat = mch_cat_tmp[qual_cuts].copy(deep=True)
gv = GroupView.build(mch_cat)

In [ ]:
len(gv.groups)

In [ ]:
from lsst.validate.drp import repeatability

In [ ]:
magkey = gv.schema.find('base_PsfFlux_mag').key
#print(magkey)
#repeatability.measurePhotRepeat('PA1','HSC-R', matches=gv, magKey=magkey)

def matchFilter(cat):
    if len(cat) < 2:
        return False
    return np.isfinite(cat.get(magkey)).all()

#repeat = repeatability.calcPhotRepeat(gv.where(matchFilter), magkey)
repeat = repeatability.measurePhotRepeat('PA1','HSC-R', matches=gv.where(matchFilter), magKey=magkey)

In [ ]:
dataset3 = "matchedCatalog"

mch_cat_repeat = []

def matchFilter(cat):
    if len(cat) < 2:
        return False
    return np.isfinite(cat.get(magkey)).all()

for p in patches:

    did={'tract':9813, 'abstract_filter':'r', 'instrument':'HSC', 'skymap':'hsc_rings_v1', 'patch':p}
    mch_cat_tmp = butler_gen3.get(dataset3, dataId=did, collections=[collectionName])

    qual_cuts = (mch_cat_tmp['base_ClassificationExtendedness_value'] < 0.5) &\
                (mch_cat_tmp['base_PsfFlux_snr'] > 50.0) & (mch_cat_tmp['base_PixelFlags_flag_saturated'] == False) &\
                (mch_cat_tmp['base_PixelFlags_flag_cr'] == False) & (mch_cat_tmp['base_PixelFlags_flag_bad'] == False) &\
                (mch_cat_tmp['base_PixelFlags_flag_edge'] == False)

    mch_cat = mch_cat_tmp[qual_cuts].copy(deep=True)
    gv = GroupView.build(mch_cat)
    
    if len(gv.where(matchFilter)) > 10:
        magkey = gv.schema.find('base_PsfFlux_mag').key
        repeat = repeatability.measurePhotRepeat('PA1','HSC-R', matches=gv.where(matchFilter), magKey=magkey)
    
        mch_cat_repeat.append(repeat.quantity.value)
    else:
        mch_cat_repeat.append(np.nan)

        
mch_cat_repeat = np.array(mch_cat_repeat)

In [ ]:
np.size(np.where(mch_cat_tmp['base_ClassificationExtendedness_value'] == 0))

In [ ]:
plt.scatter(tab_compare['nstars_patch'],\
            100.0*(tab_compare['PA1_direct']-mch_cat_repeat)/mch_cat_repeat,\
            c=tab_compare['PA1_MPT'], s=50, cmap='gist_rainbow')
plt.xlabel(r'N$_{\rm stars}$')
plt.ylabel('Percent difference')
plt.hlines(0,0,300,linestyle='--')
plt.xlim(0,140)
plt.colorbar(label='PA1_validate_drp (mmag)')
plt.minorticks_on()
plt.savefig('PA1_comp_RC2.png')
plt.show()

In [ ]:
plt.hist(tab_compare['PA1_direct'], bins=np.arange(0,50,2), color='Red', label='direct measurement', histtype='step', linewidth=4)
plt.hist(tab_compare['PA1_MPT'], bins=np.arange(0,50,2), color='Black', linestyle='--', label='metrics-pipeline-task', histtype='step', linewidth=4)
plt.hist(mch_cat_repeat, bins=np.arange(0,50,2), color='DodgerBlue', linestyle='-.', label='validate_drp', histtype='step', linewidth=4)
plt.vlines(np.nanmedian(tab_compare['PA1_direct']),0,18,linestyle=':',color='Red',label='median: '+str(np.nanmedian(tab_compare['PA1_direct'])))
plt.vlines(np.nanmedian(tab_compare['PA1_MPT']),0,18,linestyle=':',color='Black',label='median: '+str(np.nanmedian(tab_compare['PA1_MPT'])))
plt.vlines(np.nanmedian(mch_cat_repeat),0,18,linestyle=':',color='DodgerBlue',label='median: '+str(np.nanmedian(mch_cat_repeat)))
plt.xlabel('PA1 (mmag)')
plt.ylabel('N')
plt.ylim(0,18)
plt.legend()
plt.minorticks_on()
plt.savefig('PA1_comp_RC2_tract9813_hist_withValidateDrp.png')
plt.show()